In [1]:
import pandas
from bs4 import BeautifulSoup
import splinter
import re
import os

In [2]:
spells_to_import = [
    "http://dnd5e.wikidot.com/spell:prestidigitation",
    "http://dnd5e.wikidot.com/spell:minor-illusion",
    "http://dnd5e.wikidot.com/spell:mage-hand",
    "http://dnd5e.wikidot.com/spell:shield",
    "http://dnd5e.wikidot.com/spell:detect-magic",
    "http://dnd5e.wikidot.com/spell:disguise-self",
    "http://dnd5e.wikidot.com/spell:sleep",
    "http://dnd5e.wikidot.com/spell:find-familiar",
    "http://dnd5e.wikidot.com/spell:snare"
]
database = "spells_wikidot.csv"

In [3]:
browser = splinter.Browser("firefox")

In [4]:
url = spells_to_import[0]

In [22]:
 def get_spell(url):
    browser.visit(url)
    soup = BeautifulSoup(browser.html, 'html.parser')
    main_content = soup.find("div", class_="main-content")
    title = main_content.find("div", class_="page-title")
    name = title.text.strip()
    content = main_content.find("div", id="page-content")
    paragraphs = content.find_all("p")
    source = paragraphs[0].text.strip()
    school = paragraphs[1].text.strip()
    traits = paragraphs[2].text.strip().split("\n")
    casting_time = traits[0].split(":")[1].strip()
    Range = traits[1].split(":")[1].strip()
    components = traits[2].split(":")[1].strip()
    duration = traits[3].split(":")[1].strip()
    html = content.html
    return {
        "name":name,
        "source":source,
        "casting_time":casting_time,
        "range":Range,
        "components":components,
        "duration":duration,
        "html":str(content),
        "image":""
    }

In [26]:
old_spell_df = None
if os.path.exists(database):
    old_spell_df = pandas.read_csv(database).set_index('url', drop=False)
else:
    old_spell_df = pandas.DataFrame()
spell_list = []
for url in spells_to_import:
    if 'url' in old_spell_df.columns and url in old_spell_df.index:
        old_row = old_spell_df.loc[url]
        print(f"using old data for {old_row['name']}")
        spell_list.append(old_row.to_dict())
    else:
        print(f"downloading {url}")
        spell_list.append(get_spell(url))
    spell_df = pandas.DataFrame(spell_list)
    spell_df.to_csv(database, index=False)
spell_df

downloading http://dnd5e.wikidot.com/spell:prestidigitation
downloading http://dnd5e.wikidot.com/spell:minor-illusion
downloading http://dnd5e.wikidot.com/spell:mage-hand
downloading http://dnd5e.wikidot.com/spell:shield
downloading http://dnd5e.wikidot.com/spell:detect-magic
downloading http://dnd5e.wikidot.com/spell:disguise-self
downloading http://dnd5e.wikidot.com/spell:sleep
downloading http://dnd5e.wikidot.com/spell:find-familiar
downloading http://dnd5e.wikidot.com/spell:snare


,name,source,casting_time,range,components,duration,html,image
0,Prestidigitation,Source: Player's Handbook,1 action,10 feet,"V, S",Up to 1 hour,"<div id=""page-content"">\n<div class=""content-s...",
1,Minor Illusion,Source: Player's Handbook,1 action,30 feet,"S, M (a bit of fleece)",1 minute,"<div id=""page-content"">\n<div class=""content-s...",
2,Mage Hand,Source: Player's Handbook,1 action,30 feet,"V, S",1 minute,"<div id=""page-content"">\n<div class=""content-s...",
3,Shield,Source: Player's Handbook,"1 reaction, which you take when you are hit by...",Self,"V, S",1 round,"<div id=""page-content"">\n<div class=""content-s...",
4,Detect Magic,Source: Player's Handbook,1 action,Self,"V, S","Concentration, up to 10 minutes","<div id=""page-content"">\n<div class=""content-s...",
5,Disguise Self,Source: Player's Handbook,1 action,Self,"V, S",1 hour,"<div id=""page-content"">\n<div class=""content-s...",
6,Sleep,Source: Player's Handbook,1 action,90 feet,"V, S, M (a pinch of fine sand, rose petals, or...",1 minute,"<div id=""page-content"">\n<div class=""content-s...",
7,Find Familiar,Source: Player's Handbook,1 hour,10 feet,"V, S, M (10 gp worth of charcoal, incense, and...",Instantaneous,"<div id=""page-content"">\n<div class=""content-s...",
8,Snare,Source: Xanathar's Guide to Everything,1 minute,Touch,"V, S, M (30 feet of cord or rope, which is con...",Until dispelled or triggered,"<div id=""page-content"">\n<div class=""content-s...",


In [27]:
browser.quit()